# 000 Forecasting Bot

Starting from https://colab.research.google.com/drive/1_Il5h2Ed4zFa6Z3bROVCE68LZcSi4wHX?usp=sharing

## Imports

In [1]:
from IPython.display import Markdown

## Today

In [2]:
import datetime
today = str(datetime.datetime.now())[0:10]
today

'2024-07-31'

## 000_bot

### API Keys

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api
- `ASKNEWS_CLIENT_ID`, `ASKNEWS_SECRET`

In [27]:
from omegaconf import OmegaConf
token_fn = "tokens.yaml"

In [29]:
config = OmegaConf.load(token_fn)

def pr(tokens):
    print(OmegaConf.to_yaml(config))

### LLM and Metaculus Interaction

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [4]:
import datetime
import json
import os
import requests
import re
from openai import OpenAI
from tqdm import tqdm

In [5]:
AUTH_HEADERS = {"headers": {"Authorization": f"Token {config.METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3349
SUBMIT_PREDICTION = True

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Comment posted for ", question_id)

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Prediction posted for ", question_id)


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=1000):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "open",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

### IFP

In [6]:
class IFP:

    def __init__(self, question_id):
        self.question_id = question_id
        self.question_details = get_question_details(self.question_id)
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")   
        self.title = self.question_details["title"]
        self.resolution_criteria = self.question_details["resolution_criteria"]
        self.background = self.question_details["description"]
        self.fine_print = self.question_details["fine_print"]

    def report(self):
        rpt = f"""
The future event is described by this question: [ {self.title} ]
The resolution criteria are: [ {self.resolution_criteria} ]
The background is: [ {self.background} ]"""
        if self.fine_print:
            rpt += f"""
The fine print is: [ {self.fine_print} ]"""
        return rpt

### LLMs

### In general

In [7]:
class LLM:
    def __init__(self, system_role):
        self.messages = [{"role": "system", "content": system_role}]

    def chat(self, query):
        self.messages.append({"role": "user", "content": query})
        text = self.message()
        self.messages.append({"role": "assistant", "content": text})
        return text

### MetaAI

https://pypi.org/project/meta-ai-api/1.0.6/

In [20]:
 from meta_ai_api import MetaAI

In [ ]:
ai = MetaAI()
response = ai.prompt(message="Whats the weather in San Francisco today? And what is the date?")

In [24]:
response.keys()

dict_keys(['message', 'sources', 'media'])

In [ ]:
print(response['message'])

Today's date is Wednesday, July 31, 2024 ¹. Unfortunately, I wasn't able to find information on the current weather in San Francisco. However, I can suggest checking a weather website such as (link unavailable) for the most up-to-date information.
Here are some key details about today's date ²:
Day of the Week: Wednesday
Day Number: 213 of 366
Current Week Number: 30 of 52
Days Left in 2024: 153 days
Weeks Left in 2024: 21 weeks and 6 days



In [25]:
print(response['sources'])

[{'link': 'https://www.timeanddate.com/', 'title': 'Time and Date'}, {'link': 'https://www.inchcalculator.com/what-is-todays-date/', 'title': "What Is Today\\'s Date? - Inch Calculator"}]


In [26]:
print(response['media'])

[]


### HuggingChat

https://pypi.org/project/hugchat/

In [21]:
from hugchat import hugchat
from hugchat.login import Login

In [32]:
# Log in to huggingface and grant authorization to huggingchat
EMAIL = config.HUGGINGFACE_USERNAME
PASSWD = config.HUGGINGFACE_PASSWORD
cookie_path_dir = "./cookies/" # NOTE: trailing slash (/) is required to avoid errors
sign = Login(EMAIL, PASSWD)
cookies = sign.login(cookie_dir_path=cookie_path_dir, save_cookies=True)

In [33]:
# Create your ChatBot
chatbot = hugchat.ChatBot(cookies=cookies.get_dict())  # or cookie_path="usercookies/<email>.json"

message_result = chatbot.chat("Hi!") # note: message_result is a generator, the method will return immediately.

In [36]:
print(message_result)

It's nice to meet you. Is there something I can help you with or would you like to chat?


In [ ]:
# Non stream
message_str: str = message_result.wait_until_done() # you can also print(message_result) directly. 
# get files(such as images)
file_list = message_result.get_files_created() # must call wait_until_done() first!

# tips: model "CohereForAI/c4ai-command-r-plus" can generate images :)

# Stream response
for resp in chatbot.query(
    "Hello",
    stream=True
):
    print(resp)

In [ ]:
# Web search
query_result = chatbot.query("Hi!", web_search=True)
print(query_from hugchat import hugchat
from hugchat.login import Login

# Log in to huggingface and grant authorization to huggingchat
EMAIL = "your email"
PASSWD = "your password"
cookie_path_dir = "./cookies/" # NOTE: trailing slash (/) is required to avoid errors
sign = Login(EMAIL, PASSWD)
cookies = sign.login(cookie_dir_path=cookie_path_dir, save_cookies=True)

# Create your ChatBot
chatbot = hugchat.ChatBot(cookies=cookies.get_dict())  # or cookie_path="usercookies/<email>.json"

message_result = chatbot.chat("Hi!") # note: message_result is a generator, the method will return immediately.

# Non stream
message_str: str = message_result.wait_until_done() # you can also print(message_result) directly. 
# get files(such as images)
file_list = message_result.get_files_created() # must call wait_until_done() first!

# tips: model "CohereForAI/c4ai-command-r-plus" can generate images :)

# Stream response
for resp in chatbot.query(
    "Hello",
    stream=True
):
    print(resp)

# Web search
query_result = chatbot.query("Hi!", web_search=True)
print(query_result)
for source in query_result.web_search_sources:
    print(source.link)
    print(source.title)
    print(source.hostname)

# Create a new conversation
chatbot.new_conversation(switch_to = True) # switch to the new conversation

# Get conversations on the server that are not from the current session (all your conversations in huggingchat)
conversation_list = chatbot.get_remote_conversations(replace_conversation_list=True)
# Get conversation list(local)
conversation_list = chatbot.get_conversation_list()

# Get the available models (not hardcore)
models = chatbot.get_available_llm_models()

# Switch model with given index
chatbot.switch_llm(0) # Switch to the first model
chatbot.switch_llm(1) # Switch to the second model

# Get information about the current conversation
info = chatbot.get_conversation_info()
print(info.id, info.title, info.model, info.system_prompt, info.history)

# Assistant
assistant = chatbot.search_assistant(assistant_name="ChatGpt") # assistant name list in https://huggingface.co/chat/assistants
assistant_list = chatbot.get_assistant_list_by_page(page=0)
chatbot.new_conversation(assistant=assistant, switch_to=True) # create a new conversation with assistant

# [DANGER] Delete all the conversations for the logged in user
chatbot.delete_all_conversations()result)
for source in query_result.web_search_sources:
    print(source.link)
    print(source.title)
    print(source.hostname)

# Create a new conversation
chatbot.new_conversation(switch_to = True) # switch to the new conversation

# Get conversations on the server that are not from the current session (all your conversations in huggingchat)
conversation_list = chatbot.get_remote_conversations(replace_conversation_list=True)
# Get conversation list(local)
conversation_list = chatbot.get_conversation_list()

# Get the available models (not hardcore)
models = chatbot.get_available_llm_models()

# Switch model with given index
chatbot.switch_llm(0) # Switch to the first model
chatbot.switch_llm(1) # Switch to the second model

# Get information about the current conversation
info = chatbot.get_conversation_info()
print(info.id, info.title, info.model, info.system_prompt, info.history)

# Assistant
assistant = chatbot.search_assistant(assistant_name="ChatGpt") # assistant name list in https://huggingface.co/chat/assistants
assistant_list = chatbot.get_assistant_list_by_page(page=0)
chatbot.new_conversation(assistant=assistant, switch_to=True) # create a new conversation with assistant

# [DANGER] Delete all the conversations for the logged in user
chatbot.delete_all_conversations()

### Anthropic

### AskNews

https://colab.research.google.com/drive/1tc383HraMZOiyfKFF1EXAtlTYbsuv3Q5?usp=sharing

In [38]:
ASKNEWS_CLIENT_ID = config.ASKNEWS_CLIENT_ID
ASKNEWS_SECRET = config.ASKNEWS_SECRET

In [40]:
from asknews_sdk import AskNewsSDK

ask = AskNewsSDK(
      client_id=config.ASKNEWS_CLIENT_ID,
      client_secret=config.ASKNEWS_SECRET,
      scopes=["news"]
  )


query = "Effect of fed policy on tech sector"

# prompt-optimized string ready to go for any LLM:
news_context = ask.news.search_news(query).as_string

In [42]:
print(news_context)


<doc>
[1]:
Title: Tech Stocks Surge Amid Fed's Rate Hold
Summary: The S&P 500 and Nasdaq experienced their largest daily gains since February, driven by a rally in chip stocks and the Federal Reserve's decision to keep interest rates unchanged. The Fed hinted at potential rate easing in September, contingent on cooling inflation. Technology and discretionary stocks led the market, with seven of the 11 S&P 500 sectors advancing. The Dow ended the day up 0.24%, the S&P 500 gained 1.58%, and the Nasdaq Composite surged 2.64%. Federal Reserve Chair Jerome Powell stated that while the Fed discussed cutting rates, a strong majority agreed to delay the decision, citing July U.S. private payrolls data showing an unexpectedly small increase.
Source: Devdiscourse
Published: July 31 2024 21:33
Organization: Federal Reserve, Nasdaq, Dow, Fed, S&P 500
Date: February, September, July
Product: chip stocks, discretionary stocks
Technology: Technology
Number: 11, seven
Quantity: 2.64%, 0.24%, 1.58%
Pe

In [ ]:
def get_asknews_context(query):
  """
  Use the AskNews `news` endpoint to get news context for your query.
  The full API reference can be found here: https://docs.asknews.app/en/reference#get-/v1/news/search
  """
  ask = AskNewsSDK(
      client_id=ASKNEWS_CLIENT_ID,
      client_secret=ASKNEWS_SECRET,
      scopes=["news"]
  )

  # # get the latest news related to the query (within the past 48 hours)
  hot_response = ask.news.search_news(
      query=query, # your natural language query
      n_articles=5, # control the number of articles to include in the context
      return_type="both",
      strategy="latest news" # enforces looking at the latest news only
  )

  # get context from the "historical" database that contains a news archive going back to 2023
  historical_response = ask.news.search_news(
      query=query,
      n_articles=20,
      return_type="both",
      strategy="news knowledge" # looks for relevant news within the past 60 days
  )

  # you can also specify a time range for your historical search if you want to
  # slice your search up periodically.
  # now = datetime.datetime.now().timestamp()
  # start = (datetime.datetime.now() - datetime.timedelta(days=100)).timestamp()
  # historical_response = ask.news.search_news(
  #     query=query,
  #     n_articles=20,
  #     return_type="both",
  #     historical=True,
  #     start_timestamp=int(start),
  #     end_timestamp=int(now)
  # )

  llm_context = hot_response.as_string + historical_response.as_string
  formatted_articles = format_asknews_context(
      hot_response.as_dicts, historical_response.as_dicts)
  return llm_context, formatted_articles


def format_asknews_context(hot_articles, historical_articles):
  """
  Format the articles for posting to Metaculus.
  """

  formatted_articles = "Here are the relevant news articles:\n\n"

  if hot_articles:
    hot_articles = [article.__dict__ for article in hot_articles]
    hot_articles = sorted(
        hot_articles, key=lambda x: x['pub_date'], reverse=True)

    for article in hot_articles:
        pub_date = article["pub_date"].strftime("%B %d, %Y %I:%M %p")
        formatted_articles += f"**{article['eng_title']}**\n{article['summary']}\nOriginal language: {article['language']}\nPublish date: {pub_date}\nSource:[{article['source_id']}]({article['article_url']})\n\n"

  if historical_articles:
    historical_articles = [article.__dict__ for article in historical_articles]
    historical_articles = sorted(
        historical_articles, key=lambda x: x['pub_date'], reverse=True)

    for article in historical_articles:
        pub_date = article["pub_date"].strftime("%B %d, %Y %I:%M %p")
        formatted_articles += f"**{article['eng_title']}**\n{article['summary']}\nOriginal language: {article['language']}\nPublish date: {pub_date}\nSource:[{article['source_id']}]({article['article_url']})\n\n"

  if not hot_articles and not historical_articles:
    formatted_articles += "No articles were found.\n\n"
    return formatted_articles

  formatted_articles += f"*Generated by AI at [AskNews](https://asknews.app), check out the [API](https://docs.asknews.app) for more information*."

  return formatted_articles

### Perplexity

In [43]:
class Perplexity(LLM):
    def message(self):
        url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {config.PERPLEXITY_API_KEY}",
            "content-type": "application/json"  }
        payload = {"model": config.PERPLEXITY_MODEL, "messages": self.messages }
        response = requests.post(url=url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

### OpenAI

TBD: Metaculus credentials

In [44]:
class ChatGPT(LLM):
    def __init__(self, system_role):
        super().__init__(system_role)
        self.client = OpenAI(api_key=config.OPENAI_API_KEY)

    def message(self):
        chat_completion = self.client.chat.completions.create(
            model=config.OPENAI_MODEL,
            messages= self.messages)
        return chat_completion.choices[0].message.content

### Test questions

### Agents

In [45]:
class Agent:
    def __init__(self, system_role, llm):
        self.llm = llm(system_role)

    def chat(self, prompt):
        return self.llm.chat(prompt)

## Rate Analyzer(TBD: ChatGPT is better)

#### Researcher

In [46]:
class Researcher(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are an open source intelligence analyst.
You summarize news related to questions about events.
A question about an event is formatted as |id|question|criteria|background|fineprint|.
You will find on the web and report any reliable information you can gather about the question.
Do not make an assessment of probability.
Do not repeat information provided to you already in the prompt."""
        super().__init__(self.system_role, llm)

    def recherche(self, ifp):
        prompt = f"|{ifp.question_id}|{ifp.title}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.R = self.chat(prompt)
        ifp.news = self.R
        print(ifp.question_id, ifp.title)
        print(self.R)

    def research(self, ifps):
        for ifp in tqdm(ifps):
            self.recherche(ifps[ifp])

### Question relator

In [47]:
class QuestionRelator(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are prompted with list of forecasting questions, each with an id and a title.
Label each question with an underlying event.
If the questions are for the same event, use the same name for the underlying event.
Please return as separate lines formatted as |event|id|title|, do not add any other formatting.
"""
        super().__init__(self.system_role, llm)

    def relate(self, ifps):
        prompt = '\n'.join([f"{ifp.question_id}: {ifp.title}" for ifp in ifps.values()])
        KL = self.chat(prompt)
        K1 = [x.split('|') for x in KL.split('\n')]
        K2 = [(int(id),event) for _,event,id,_,_ in K1]
        for id,event in K2:
            ifps[id].event = event
            print(id, event)

#### Superforecaster

In [48]:
class Superforecaster(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are a superforecaster.  
You assign a probability to questions about events.
Questions are given as separate groups of lines formatted as |FORECAST|event|id|question|news|criteria|background|fineprint|.
Groups are separated by '^^^'.
Questions which are about the same event should be assigned consistent probabilities.
Reply to questions with |ASSESSMENT|id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and rationale is your reasoning for the forecast.
Separate each question with '^^^'.
Do not add any additional headings or group labels or other formatting.
After your initial forecast you may receive feedback of form |CRITIC|id|feedback|.
Reply to each feedback with |id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and 
and rationale is a revised assessment which may be adjusted from a prior assessment due the feedback unless the feedback is "I concur".
"""
        super().__init__(self.system_role, llm)

    def forecast(self, ifps):
        prompt = '^^^'.join([f"FORECAST|{ifp.event}|{ifp.question_id}|{ifp.news}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|" for ifp in ifps.values()])
        self.F0 = self.chat(prompt)
        self.F1 = [x.strip().replace('\n', '') for x in self.F0.split('^^^')]
        self.F2 = [x.split('|') for x in self.F1] 
        self.F3 = [[x for x in y if x] for y in self.F2]
        self.F4 = [(int(id),int(forecast),rationale) for _, id, forecast, rationale in self.F3]
        for id, forecast, rationale in self.F4:
            ifps[id].forecast = forecast
            ifps[id].rationale = rationale
            print(id, forecast, rationale)

    def reassess(self, ifp):
        prompt = f"|CRITIC|{ifp.question_id}|{ifp.feedback}|"
        self.R0 = self.chat(prompt)
        id,fcst,rationale = [x for x in self.R0.strip().split('|') if x]
        id = int(id)
        fcst = int(fcst)
        ifps[id].forecast = fcst
        ifps[id].rationale = rationale
        print(id, fcst, rationale)

#### Critic

In [49]:
class Critic (Agent):
    def __init__(self, llm):

        self.system_role = f"""
You a very smart and worldly person reviewing a superforecaster's assignment of probabilities to events.
You will receive an event with probabilities given as |event|id|question|zz|rationale|news|criteria|background|fineprint|.
zz is an integer probability from 1 to 99 and rationale is the student's logic for assigning probability of zz.
You will reply with a line |id|feedback| where feedback is "I concur" if you see no problem with the rationale and zz otherwise presents possible problems with the rationale and zz.
"""
        super().__init__(self.system_role, llm)

    def feedback(self, ifp):
        prompt = f"|{ifp.event}|{ifp.question_id}|{ifp.title}|{ifp.forecast}|{ifp.rationale}|{ifp.news}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.fb = self.chat(prompt)
        self.fb1 = self.fb.split('|')
        self.fb2 = [x for x in self.fb1 if x]
        try:
            id,feedback = self.fb2
            ifps[int(id)].feedback = feedback
            print(id, feedback)
        except:
            print('problem', self.fb2)
            ifps[int(id)].feedback = 'I concur'

## Summarizer

Add an agent to summarize the back and forth between critic and forecaster into a single cogent rationale that incorporates all that was discussed.

### Forecasting process

In [50]:
def forecasting(ifps):
    max_tries = 4
    
    analyst = Researcher(ChatGPT)
    analyst.research(ifps)
    
    qr = QuestionRelator(ChatGPT)
    qr.relate(ifps)
    
    sf = Superforecaster(ChatGPT)
    sf.forecast(ifps)
    
    for ifp in ifps.values():
        print("Refining", ifp.question_id)
        ifp.feedback = ''
        critic = Critic(Perplexity)
        for i in range(max_tries):
            print("Pass", i, "of", max_tries, "on", ifp.question_id)
            if 'I concur' in ifp.feedback:
                break
            critic.feedback(ifp)
            if 'I concur' in ifp.feedback:
                break
            sf.reassess(ifp)
        print("===============================================")

    for ifp in ifps.values():
        print(ifp.question_id, ifp.title)
        print("Forecast", ifp.forecast)
        print("Rationale", ifp.rationale, '\n')

In [51]:
def upload(ifp):
    post_question_prediction(ifp.question_id, ifp.forecast)
    post_question_comment(ifp.question_id, ifp.rationale)

In [52]:
def uploads(ifps):
    for ifp in ifps.values():
        upload(ifp)

## Daily forecast

### Get IFP ids

In [53]:
ifps = list_questions()['results']
today_ids = list(sorted([x['id'] for x in ifps]))
# today_ids = [25876, 25877, 25875, 25873, 25871, 25878, 25874, 25872] # 08JUL24
# today_ids = [26006, 25936, 25935, 25934, 25933, 26004, 26005] # 09JUL24
# today_ids = [25955, 25956, 25957, 25960, 25959, 25954, 25953, 25952, 25958] # 10JUL24
# today_ids = [26019, 26018, 26017, 26020, 26022, 26021, 26023, 26024] # 11JUL24
# today_ids = [26095, 26096, 26097, 26098, 26099, 26100, 26101, 26102] # 12JUL24
# today_ids = [26133, 26134, 26138, 26139, 26140, 26157, 26158, 26159] # 15JUL24
# today_ids = [26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196] # 16JUL24
# today_ids = [26210, 26211, 26212, 26213, 26214, 26215, 26216] # 17JUL24
# today_ids = [26232, 26233, 26234, 26235, 26236] # 18JUL24
# today_ids = [26302, 26303, 26304, 26305, 26306, 26307] # 19JUL24
# today_ids = [26387, 26388, 26389, 26390, 26391, 26392] # 22JUL24
# today_ids = [26404, 26405, 26406, 26407, 26408] # 23JUL24
# today_ids = [26550, 26551, 26552, 26553, 26554, 26555] # 24JUL24
# today_ids = [26568, 26569, 26570, 26571, 26572, 26573, 26574, 26575, 26576, 26577] # 25JUL24
# today_ids = [26638, 26639, 26640, 26641, 26642, 26643, 26644, 26645, 26646] # 26JUL24
# today_ids = [26665, 26666, 26667, 26668, 26669, 26670, 26671, 26683] # 29JUL24
# today_ids = [26700, 26701, 26702, 26703, 26704, 26705, 26706] # 30JUL24
# today_ids = [26816, 26817, 26818, 26819, 26820, 26821, 26844] # 31JUL24

In [54]:
today_ids

[26816, 26817, 26818, 26819, 26820, 26821, 26844]

## Forecast

In [55]:
ifps = {id: IFP(id) for id in today_ids}

In [56]:
forecasting(ifps)

 14%|█████████████████▌                                                                                                         | 1/7 [00:04<00:26,  4.36s/it]

26816 Before October 1, 2024, will Donald Trump and Kamala Harris shake hands in any of their debates, if any happen?
To address whether Donald Trump and Kamala Harris might shake hands during a potential debate before October 1, 2024, let's review the current information:

1. **Official Debates Scheduled**: No official debates between Donald Trump and Kamala Harris have been scheduled as of now. With Joe Biden having suspended his candidacy, Kamala Harris is a potential Democratic candidate, but there has been no formal confirmation of debates.

2. **Historical Handshake Practices**: Historically, Trump has shown varying approaches. He shook hands with Hillary Clinton in two out of three debates in 2016 but avoided handshakes with Joe Biden in 2020 due to COVID-19 protocols. Kamala Harris, as the running mate of Biden in 2020, also did not partake in handshakes with her debate opponent, Mike Pence, due to the same reason.

3. **Personal Relations and Public Statements**:
    - Trump p

 29%|███████████████████████████████████▏                                                                                       | 2/7 [00:11<00:30,  6.01s/it]

26817 Will the UK Abolish the Two-Child Benefit Cap before October 1, 2024?
To determine if the UK will abolish the two-child benefit cap before October 1, 2024, let's assess the current landscape and any recent developments.

1. **Current Government Position**: The Labour government, led by Prime Minister Keir Starmer, has opposed scrapping the cap, citing financial restraints. This stance is significant since it points to a reluctance to abolish the cap despite the visible criticisms and pressures.

2. **Internal Party Dynamics**: Recently, Labour suspended seven MPs who defied the party line on this issue, underscoring internal divisions and the significant political complexity around removing the cap.

3. **Public Opinion**: A YouGov poll indicates that 60% of the general UK public, including 50% of Labour supporters, oppose lifting the cap. Public sentiment appears to be against removing the cap, which may influence political decisions.

4. **Potential Legislative Paths**:
    - T

 43%|████████████████████████████████████████████████████▋                                                                      | 3/7 [00:23<00:35,  8.95s/it]

...

 86%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▍                 | 6/7 [00:47<00:08,  8.10s/it]

26821 Will there be a debate between the Republican vice presidential nominee and the Democratic vice presidential nominee before October 1, 2024?
To determine whether there will be a debate between the Republican and Democratic vice presidential nominees before October 1, 2024, we need to look at the current state of political affairs and recent trends in organizing VP debates:

1. **Current Political Context**: 
    - **Republican VP Nominee**: JD Vance has been announced as the Republican vice presidential nominee.
    - **Democratic VP Nominee**: The Democratic nominee for VP has not yet been announced following President Joe Biden's withdrawal from the race. The Democratic candidates' field is still in flux.

2. **Historical Precedents**:
    - Vice presidential debates are a long-standing tradition in U.S. presidential elections. There was a single VP debate in the 2020 election cycle.
    - VP debates have typically been organized by the Commission on Presidential Debates (CPD).

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:53<00:00,  7.69s/it]

26844 Will the opposition candidate, Edmundo González, concede the 2024 Venezuelan election to Maduro before September 2024?
To determine whether Edmundo González will publicly concede the 2024 Venezuelan election to Nicolás Maduro before September 1, 2024, we need to observe the current political situation and any statements or actions by González:

1. **Election Context**:
    - Nicolás Maduro was declared the winner of the July 28, 2024, Venezuelan presidential election with 51.2% of the vote.
    - Opposition candidate Edmundo González received 44.2% of the vote.
    - The election has been widely reported as marred by irregularities, and international observers have questioned the results.

2. **González's Position**:
    - As of now, González has not conceded the election to Maduro.
    - Historical context and the current political climate in Venezuela suggest that conceding might be challenging due to irregularities and the opposition's usual stance against Maduro's regime.
   

26816 2024 Trump-Harris debates and shake hands
26817 UK Two-Child Benefit Cap
26818 Donald Trump's TikTok followers
26819 Kamala Harris's TikTok followers
26820 Robert F. Kennedy Jr.'s TikTok followers
26821 2024 Vice Presidential debate
26844 2024 Venezuelan election
26816 40 The absence of scheduled presidential debates and the lack of formal announcement about debates between Trump and Harris, combined with historical precedent and current hostile relations, makes it relatively improbable that a handshake will occur. However, mutual interest in debates might foster unexpected dynamics.
26817 20 The Labour government's resistance to abolishing the two-child benefit cap, opposition within the party, and significant public disapproval indicate a low likelihood of policy change before October 1, 2024. Legislative processes and political dynamics add complexity to the issue.
26818 80 Given Trump's rapid follower growth rate and the trend from 9.3 million, it appears quite plausible he w

## Upload

In [57]:
uploads(ifps)

Prediction posted for  26816
Comment posted for  26816
Prediction posted for  26817
Comment posted for  26817
Prediction posted for  26818
Comment posted for  26818
Prediction posted for  26819
Comment posted for  26819
Prediction posted for  26820
Comment posted for  26820
Prediction posted for  26821
Comment posted for  26821
Prediction posted for  26844
Comment posted for  26844


In [58]:
len(today_ids)

7